# Sentinel Hub data download

We can download imagery data in Sentinel Hub through the EOTDL client.

Uncomment the following line to install eotdl if needed.

In [9]:
# !pip install eotdl

Initialize the SH client.

In [18]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [19]:
from eotdl.access.sentinelhub.client import SHClient

client = SHClient()

Let's download Sentinel-2 imagery in Jaca, Spain, between 1 to 10 January, 2020.

In [20]:
dates = ('2020-01-01', '2020-01-10')
jaca_bounding_box = (-0.557034,42.569409,-0.547706,42.57571)

In [21]:
search_dict = {
    'Jaca': {
        'bounding_box': jaca_bounding_box,
        'time_interval': dates
    }
}

Some explanation about the following code block. In order to download imagery through the Sentinel Hub API, we need to define some parameters, such as the Data Collection (in this workshop, `sentinel-2-l2a`), the EvalScript, the resolution and so on. In order to avoid this time-expensive definition, we have wrapped those parameters in a Python class like `sentinel_2_l2a_download_parameters`. With that, we just have to give the download dict with the bounding box and time interval and the folder where we want to download the data. And voilà! All it's managed by the EOTDL environment.

Arrived to this point, and as we have seen in the [previous notebook](11_sh_search.ipynb), we can pass a simple search dictionary, so that all images available in the date range will be downloaded, or the dictionary returned from the `get_available_data_by_location` function, so that the images of the indicated dates will be downloaded.

In this case, we will let the EOTDL look and download for all the available images.

In [22]:
from eotdl.access import sentinel_2_l2a_download_parameters

sentinel_2_l2a_download_parameters.data_to_download = search_dict   # Give the dictionary with the data to download
sentinel_2_l2a_download_parameters.data_folder = 'data/sentinel_2'  # Give the folder where the data will be downloaded

process_requests = client.request_bulk_data(sentinel_2_l2a_download_parameters)

In [23]:
%%time

data = client.download_data(process_requests)

CPU times: user 69.3 ms, sys: 15.3 ms, total: 84.6 ms
Wall time: 831 ms


That's all! We have downloaded the images for our dataset. Let's check them!

In [24]:
from glob import glob

rasters = glob('data/sentinel_2/*/*/*.tiff')
rasters

['data/sentinel_2/Jaca_2020-01-09/3edb5693f0d781595997bffd04b92a67/response.tiff',
 'data/sentinel_2/Jaca_2020-01-01/154f85cf8673b548ddbf1f2eb1860754/response.tiff',
 'data/sentinel_2/Jaca_2020-01-06/c95084396569cc69f4673e178f8cc8bf/response.tiff',
 'data/sentinel_2/Jaca_2020-01-04/81126e4ec8a68f126c48c3fea527071b/response.tiff']

What if we want to see a random image? We can display it and explore it interactively using [leafmap](https://leafmap.org/).

In [25]:
import leafmap.leafmap as leafmap
from random import choice

m = leafmap.Map()
raster = choice(rasters)
m.add_raster(raster, bands=[5, 4, 3], layer_name='Raster')
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…